In [1]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq


load_dotenv()
groq_api=os.getenv('groq_api')
llm=ChatGroq(model='gemma2-9b-it',api_key=groq_api)


## Runnables Seq

In [3]:
from langchain.schema.runnable import RunnableSequence


parser = StrOutputParser()

prompt1 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)
prompt2 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)
chain=RunnableSequence(prompt1,llm,parser,prompt2,llm,parser)
chain.invoke({'topic':'Gravity'})

'You got it! This joke plays on the double meaning of the phrase "make up."\n\n* **Literal Meaning:** Atoms are the fundamental building blocks of all matter. They literally "make up" everything around us, from the chairs we sit on to the air we breathe. \n\n* **Figurative Meaning:**  "Making up" something can also mean to invent a story or lie. \n\nThe joke sets you up to think about trustworthiness, then uses the scientific fact about atoms to deliver a punny twist.  It\'s funny because it\'s unexpected and uses wordplay.  \n\n\nWant to hear another joke?  😄 \n\n'

## Runnable Parallel

In [5]:

from langchain.schema.runnable import RunnableParallel,RunnableSequence

prompt1 = PromptTemplate(
    template='Generate a tweet about {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Generate a Linkedin post about {topic}',
    input_variables=['topic']
)



parser = StrOutputParser()

parallel_chain = RunnableParallel({
    'tweet': RunnableSequence(prompt1, llm, parser),
    'linkedin': RunnableSequence(prompt2, llm, parser)
})

result = parallel_chain.invoke({'topic':'AI'})

print(result['tweet'])
print(result['linkedin'])

AI is changing the world at an incredible pace! 🤯  From writing code to creating art, its potential seems limitless. What do you think the biggest impact of AI will be in the next 5 years? #AI #technology #future  🚀

##  Is AI a Threat or an Opportunity? 🤔

The rapid advancements in AI are undeniable, sparking both excitement and apprehension.  

While some fear job displacement and ethical dilemmas, I see AI as a powerful tool for **enhancing our capabilities and driving positive change**. 

🚀 **Here's why:**

* **Increased efficiency and productivity:** AI can automate repetitive tasks, freeing up human time for more creative and strategic endeavors.
* **Data-driven insights:** AI can analyze massive datasets to reveal hidden patterns and trends, leading to better decision-making.
* **Personalized experiences:** AI can tailor products and services to individual needs, creating more engaging and satisfying interactions.

**The key to harnessing the potential of AI lies in responsible 

## Runnables Passthrough

In [10]:
from langchain.schema.runnable import RunnablePassthrough,RunnableSequence,RunnableParallel

prompt11 = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

prompt21 = PromptTemplate(
    template='Explain the following joke - {text}',
    input_variables=['text']
)

joke_chain=RunnableSequence(prompt11|llm|parser)

parelle_chain=RunnableParallel(
    {
    'joke':RunnablePassthrough(),
    'explain':RunnableSequence(prompt21, llm, parser)
    }
)

final_chain=RunnableSequence(joke_chain, parallel_chain)

print(final_chain.invoke({'topic':'cricket'}))

{'tweet': 'Why did the cricket player bring a ladder to the match? 🪜🏏 Because he heard the scores were high! 😂 #cricketjokes #dadjokes \n\nLet me know if you want another one! 😊  \n\n', 'linkedin': "## Why did the cricket player bring a ladder to the match? 🪜🏏\n\nBecause he heard the scores were high! 😂 \n\nThis one's a real wicket-taker!  \n\nLet's be honest, sometimes all it takes is a good laugh to get us through the day.  What's your favorite cricket joke? Share it in the comments below! 👇\n\n#cricket #jokes #humour #sports #cricketlife #laughterisbestmedicine  \n\n\n\n"}


## Runnable Lambda

In [11]:
from langchain.schema.runnable import RunnableSequence, RunnableLambda, RunnablePassthrough, RunnableParallel

def word_count(text):
    return len(text.split())

prompt = PromptTemplate(
    template='Write a joke about {topic}',
    input_variables=['topic']
)

parser = StrOutputParser()
joke_gen_chain = RunnableSequence(prompt, llm, parser)

parallel_chain = RunnableParallel({
    'joke': RunnablePassthrough(),
    'word_count': RunnableLambda(word_count)
})

final_chain = RunnableSequence(joke_gen_chain, parallel_chain)

result = final_chain.invoke({'topic':'AI'})

final_result = """{} \n word count - {}""".format(result['joke'], result['word_count'])

print(final_result)

Why was the AI so good at poker? 

Because it had a neural network!  🃏🧠  


Let me know if you'd like to hear another one! 😊  
 
 word count - 26


## Runnable Branch

In [14]:
from langchain.schema.runnable import RunnableSequence, RunnableParallel, RunnablePassthrough, RunnableBranch, RunnableLambda

prompt1 = PromptTemplate(
    template='Write a detailed report on {topic}',
    input_variables=['topic']
)

prompt2 = PromptTemplate(
    template='Summarize the following text \n {text}',
    input_variables=['text']
)

In [15]:
parser = StrOutputParser()

report_gen_chain = prompt1 | llm| parser

branch_chain = RunnableBranch(
    (lambda x: len(x.split())>300, prompt2 | llm | parser),
    RunnablePassthrough()
)

final_chain = RunnableSequence(report_gen_chain, branch_chain)

print(final_chain.invoke({'topic':'Russia vs Ukraine'}))


This report provides a comprehensive analysis of the Russia-Ukraine conflict, tracing its roots back to the collapse of the Soviet Union and highlighting key events like the Orange Revolution, the Euromaidan Uprising, and the 2014 annexation of Crimea.  

The report details the conflict's political and economic ramifications for both Ukraine and Russia, as well as its impact on NATO and the global order. 

It emphasizes the international condemnation and economic sanctions imposed on Russia following its full-scale invasion of Ukraine in 2022. 

Finally, the report outlines potential future scenarios, ranging from a protracted conflict to a negotiated settlement or even a wider war involving NATO, stressing the uncertainty surrounding the conflict's outcome and its lasting geopolitical implications. 




